In [1]:
# External
import sys
from tensorflow import keras
import numpy as np
## My lib
sys.path.append('../src/')
from dataset.data_utils import get_data
from models.generate_results_process import process_rnn
import pandas as pd

2024-02-13 01:43:14.136518: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-13 01:43:14.136540: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-13 01:43:14.136560: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-13 01:43:14.143138: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-13 01:43:15.422829: W tensorflow/compiler/

## configs

In [2]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [3]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
      keras.metrics.CategoricalAccuracy(name="categorical_accuracy", dtype=None)
      ]

2024-02-13 01:43:17.893632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 01:43:17.922216: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-13 01:43:17.922404: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# constants
validation_split = 0.1
metrics = METRICS
epochs = 1000
batch_size=128

## Generate results

### Generate embedding layer Results

In [ ]:
data = get_data('../data/raw/authors.csv')

In [ ]:
vectorization = 'embedding_layer'

In [ ]:
# list of "complete rnn_layers" 
list_complete_rnn = ['GRU', 'LSTM', 'BIGRU', 'BILSTM', 'Simple']

list_complete_clf_type = []

# this variable enable generate from a start point (useful for cases that the kernel dies or the gpu memory runout)
# 0 generate all results 
start_in = 0

for rnn_layer in ['GRU','LSTM','BIGRU', 'BILSTM','Simple','BISimple']:
    
    if (rnn_layer not in list_complete_rnn):
    
        for classification_type in ['one-vs-one', 'one-vs-all', 'multiclass']:
            
            
        
                if (classification_type not in list_complete_clf_type): 
                    
                    print(f'Running {classification_type} - {rnn_layer} ...')

                    process_rnn(
                    data,
                    rnn_layer,
                    vectorization,
                    classification_type = classification_type,
                    metrics = METRICS,
                    start_in = start_in,
                    epochs = epochs,
                    batch_size=batch_size,
                    validation_split=validation_split)
                    
                    start_in = 0
                    
                else:
                    print(f'Skipped {classification_type}')
                    
        list_complete_clf_type = [] 
                    
    else:
        
        print(f'Skipped {rnn_layer}')
                  
            
    

### Generate BERT Embedding Results

In [5]:
batch_size=32

In [6]:
model_version = 'base'
max_lenght = 100 

X = np.load(f'../data/processed/bert-{model_version}-portuguese-cased_full__max_lenght={max_lenght}.npy')
y = get_data('../data/raw/authors.csv')[['username']].reset_index(drop = True)
created_utc = get_data('../data/raw/authors.csv')[['created_utc']].reset_index(drop = True)

In [7]:
vectorization = 'bert_Embedding'

In [8]:
# list of "complete rnn_layers" 
list_complete_rnn = []

list_complete_clf_type = []

# this variable enable generate from a start point (useful for cases that the kernel dies or the gpu memory runout)
# 0 generate all results 
start_in = 1


for rnn_layer in [
    #'GRU',
    # 'LSTM',
    # 'BIGRU', 
    # 'BILSTM', 
    # 'Simple',
    # 'BISimple',
    #'double-BILSTM',
    'double-BIGRU'
    ]:
    
    for classification_type in [
        #'one-vs-one', 
        #'multiclass',
        'one-vs-all', 
        ]:
    
        if (rnn_layer not in list_complete_rnn) and (classification_type not in list_complete_clf_type): 
            
            print(f'Running {classification_type} - {rnn_layer} ...')

        
            batch_size=64

            process_rnn(
            (X,y,created_utc),
            rnn_layer,
            vectorization,
            classification_type = classification_type,
            metrics = METRICS,
            start_in = start_in,
            epochs = epochs,
            batch_size=batch_size,
            validation_split=validation_split)
            
            start_in = 0 
            
        else:
            
            print(f'Skipped {classification_type} - {rnn_layer}')
            
    list_complete_clf_type = []     

Running one-vs-all - double-BIGRU ...
author 9-Cortes (0) was skipped
author AgnaldoTeExplode (1) running ...


2024-02-13 01:52:53.684066: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3009331200 exceeds 10% of free system memory.
2024-02-13 01:54:17.010698: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3009331200 exceeds 10% of free system memory.


Epoch 1/1000


2024-02-13 01:54:25.822649: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-13 01:54:26.735666: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x783bd80bf7b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-13 01:54:26.735692: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce GTX 1660 Ti, Compute Capability 7.5
2024-02-13 01:54:26.757993: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-13 01:54:26.905933: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


154/154 [==============================] - 23s 90ms/step - loss: 0.5620 - tp: 6679.0000 - fp: 3117.0000 - tn: 6679.0000 - fn: 3117.0000 - precision: 0.6818 - recall: 0.6818 - auc: 0.7529 - prc: 0.7603 - categorical_accuracy: 0.6818 - val_loss: 0.4485 - val_tp: 840.0000 - val_fp: 249.0000 - val_tn: 840.0000 - val_fn: 249.0000 - val_precision: 0.7713 - val_recall: 0.7713 - val_auc: 0.8684 - val_prc: 0.8757 - val_categorical_accuracy: 0.7713
Epoch 2/1000
154/154 [==============================] - 6s 41ms/step - loss: 0.4559 - tp: 7408.0000 - fp: 2388.0000 - tn: 7408.0000 - fn: 2388.0000 - precision: 0.7562 - recall: 0.7562 - auc: 0.8543 - prc: 0.8606 - categorical_accuracy: 0.7562 - val_loss: 0.4149 - val_tp: 868.0000 - val_fp: 221.0000 - val_tn: 868.0000 - val_fn: 221.0000 - val_precision: 0.7971 - val_recall: 0.7971 - val_auc: 0.8890 - val_prc: 0.8945 - val_categorical_accuracy: 0.7971
Epoch 3/1000
154/154 [==============================] - 6s 42ms/step - loss: 0.4008 - tp: 7880.0000 - 

2024-02-13 01:58:22.778845: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1116672000 exceeds 10% of free system memory.
2024-02-13 01:58:29.290569: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1116672000 exceeds 10% of free system memory.


114/114 [==============================] - 4s 18ms/step
author BeatoSalut (2) running ...


2024-02-13 02:11:39.723140: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 3009331200 exceeds 10% of free system memory.


Epoch 1/1000
154/154 [==============================] - 22s 95ms/step - loss: 0.6271 - tp: 7918.0000 - fp: 2967.0000 - tn: 7918.0000 - fn: 2967.0000 - precision: 0.7274 - recall: 0.7274 - auc: 0.7693 - prc: 0.7574 - categorical_accuracy: 0.7274 - val_loss: 0.5620 - val_tp: 783.0000 - val_fp: 306.0000 - val_tn: 783.0000 - val_fn: 306.0000 - val_precision: 0.7190 - val_recall: 0.7190 - val_auc: 0.7924 - val_prc: 0.7952 - val_categorical_accuracy: 0.7190
Epoch 2/1000
154/154 [==============================] - 6s 41ms/step - loss: 0.5283 - tp: 7198.0000 - fp: 2598.0000 - tn: 7198.0000 - fn: 2598.0000 - precision: 0.7348 - recall: 0.7348 - auc: 0.8220 - prc: 0.8234 - categorical_accuracy: 0.7348 - val_loss: 0.5456 - val_tp: 757.0000 - val_fp: 332.0000 - val_tn: 757.0000 - val_fn: 332.0000 - val_precision: 0.6951 - val_recall: 0.6951 - val_auc: 0.7904 - val_prc: 0.7978 - val_categorical_accuracy: 0.6951
Epoch 3/1000
154/154 [==============================] - 6s 39ms/step - loss: 0.4706 - tp: